In [1]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import bitstring
from qiskit_aer import AerSimulator

from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)

from qiskit.circuit.library import UnitaryGate

In [13]:
#https://link.springer.com/article/10.1007/s11128-020-02735-0 , https://www.researchgate.net/figure/The-encoding-and-correction-circuits-of-the-7-1-3-code-29-a-An-unknown-logical_fig7_342818848
def code(n: int):         #encodes |00>_L
    qr = QuantumRegister(7*n+2,"q")
    cbits = ClassicalRegister(14,"c")
    qc = QuantumCircuit(qr, cbits)

    for i in range(n):
        qc.h(0+7*i)
        qc.h(1+7*i)
        qc.h(2+7*i)

        qc.cx(0+7*i,3+7*i)
        qc.cx(0+7*i,5+7*i)
        qc.cx(0+7*i,6+7*i)

        qc.cx(1+7*i,3+7*i)
        qc.cx(1+7*i,4+7*i)
        qc.cx(1+7*i,6+7*i)

        qc.cx(2+7*i,4+7*i)
        qc.cx(2+7*i,5+7*i)
        qc.cx(2+7*i,6+7*i)

    return qc

def X_L(qc: QuantumCircuit, pos: int):
    for i in range(7):
        qc.x(i+7*pos)

def Z_L(qc: QuantumCircuit, pos: int):
    for i in range(7):
        qc.z(i+7*pos)
    
def H_L(qc: QuantumCircuit, pos: int):
    for i in range(7):
        qc.h(i+7*pos)

def S_L(qc: QuantumCircuit, pos: int):
    for i in range(7):
        qc.s(i+7*pos)

def adj_S_L(qc: QuantumCircuit, pos: int):
    for i in range(7):
        qc.sdg(i+7*pos)

def CNOT_L(qc: QuantumCircuit, control: int):
    if control == 0:
        for i in range(7):
            qc.cx(i, i+7)
    else:
        for i in range(7):
            qc.cx(i+7,i)

def T_L(qc: QuantumCircuit, pos: int):
    magic = qc.num_qubits - 1
    anc = magic - 1

    qc.h(magic)
    qc.t(magic)

    for i in range(7):
        qc.cx(i+7*pos,magic)

    qc.measure(magic,0)

    for i in range(7):
        with qc.if_test((0,0)):
            qc.s(i+7*pos)

    qc.reset(magic)

def adj_T_L(qc: QuantumCircuit, pos: int):
    magic = qc.num_qubits - 1
    anc = magic - 1

    qc.h(magic)
    qc.tdg(magic)

    for i in range(7):
        qc.cx(i+7*pos,magic)

    qc.measure(magic,0)

    for i in range(7):
        with qc.if_test((0,0)):
            qc.sdg(i+7*pos)

    qc.reset(magic)

def readout(qc: QuantumCircuit, shots: int):
    for i in range(qc.num_qubits-2):
        qc.measure(i,i)
    sim = AerSimulator()
    job = sim.run(qc, shots=shots)
    result = job.result()
    counts = result.get_counts()

    ehm = list(counts.keys())
    hmm = list(counts.values())
    
    if qc.num_qubits == 16:
        ehm_0 , ehm_1 , hmm_0 , hmm_1 = [],[],[],[]
        for s in ehm:
            ehm_0.append(s[:len(s)//2])
            ehm_1.append(s[len(s)//2:])
        for i in range(len(ehm_0)):
            if ehm_0[i] == hmm[i][:len(hmm[i])//2]:
                hmm_0.append(hmm[i])


    ones = 0

    for i in ehm:
        ones = i.count("1")
    
    ones = ones%2

    if ones == 0:



    return counts


In [14]:
qc = code(1)

X_L(qc,0)

readout(qc, 100)

['00000001111111', '00000000010110', '00000001001100', '00000000100101', '00000001100010', '00000001010001', '00000000001011', '00000000111000']
[17, 9, 18, 9, 10, 17, 7, 13]


{'00000001111111': 17,
 '00000000010110': 9,
 '00000001001100': 18,
 '00000000100101': 9,
 '00000001100010': 10,
 '00000001010001': 17,
 '00000000001011': 7,
 '00000000111000': 13}

In [17]:
s = "Test"

first_half  = s[:len(s)//2]
second_half = s[len(s)//2:]

print(first_half)
print(second_half)

Te
st
